In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("yarn").config('spark.executor.instances','100').appName("chiang_ct").getOrCreate()

In [3]:
%time df = spark.read.csv("/user/spark/share/splitdata.csv",header=True)

CPU times: user 4.82 ms, sys: 146 µs, total: 4.97 ms
Wall time: 9.97 s


In [4]:
df.createOrReplaceTempView('behavior')

In [5]:
%time df_pv = spark.sql('select user_cate as user_cate_pv, count(*) as pv from behavior where btag = "pv" group by user_cate')
%time df_cart = spark.sql('select user_cate as user_cate_cart, count(*) as cart from behavior where btag = "cart" group by user_cate')
%time df_fav = spark.sql('select user_cate as user_cate_fav, count(*) as fav from behavior where btag = "fav" group by user_cate')
%time df_buy = spark.sql('select user_cate as user_cate_buy, count(*) as buy from behavior where btag = "buy" group by user_cate')

CPU times: user 1.11 ms, sys: 0 ns, total: 1.11 ms
Wall time: 64.5 ms
CPU times: user 500 µs, sys: 0 ns, total: 500 µs
Wall time: 6.79 ms
CPU times: user 536 µs, sys: 0 ns, total: 536 µs
Wall time: 15.3 ms
CPU times: user 498 µs, sys: 0 ns, total: 498 µs
Wall time: 11.3 ms


In [ ]:
%time df_pv.show() #just checking

### join to cross_table

In [6]:
df_user_cate = spark.sql("select distinct user_cate from behavior") 

In [8]:
df_user_cate.select('user_cate').distinct().count() #just checking

In [9]:
#join to crosstable for table 1 
tb1_cross_all = df_user_cate.join(df_pv,(df_user_cate.user_cate == df_pv.user_cate_pv) ,'outer')
tb1_cross_all = tb1_cross_all.join(df_cart,(tb1_cross_all.user_cate == df_cart.user_cate_cart) ,'outer')
tb1_cross_all = tb1_cross_all.join(df_fav,(tb1_cross_all.user_cate == df_fav.user_cate_fav) ,'outer')
tb1_cross_all = tb1_cross_all.join(df_buy,(tb1_cross_all.user_cate == df_buy.user_cate_buy) ,'outer')

In [10]:
tb1_cross_all.columns #just for check

['user_cate',
 'user_cate_pv',
 'pv',
 'user_cate_cart',
 'cart',
 'user_cate_fav',
 'fav',
 'user_cate_buy',
 'buy']

In [11]:
tb1_cross_all = tb1_cross_all.drop('user_cate_pv','user_cate_cart','user_cate_fav','user_cate_buy')

In [12]:
tb1_cross_all.columns #just for check

['user_cate', 'pv', 'cart', 'fav', 'buy']

In [13]:
%time tb1_cross_all.write.csv("hdfs:///user/spark/share/output/table1/cross_all",header="true")

CPU times: user 90.4 ms, sys: 31.4 ms, total: 122 ms
Wall time: 14min 49s


In [16]:
tb1_cross_all.createOrReplaceTempView('be')

In [17]:
%time spark.sql("select sum(pv) as sum_pv, sum(cart) as sum_cart, sum(fav) as sum_fav, sum(buy) as sum_buy from be").show()

+---------+--------+-------+-------+
|   sum_pv|sum_cart|sum_fav|sum_buy|
+---------+--------+-------+-------+
|688835878|15946033|9301837|9115919|
+---------+--------+-------+-------+

CPU times: user 78.7 ms, sys: 26 ms, total: 105 ms
Wall time: 13min 20s


In [20]:
688835878+15946033+9301837+9115919

723199667